In [2]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'E:\\Software\\Spark\\spark-3.0.0-preview2-bin-hadoop2.7'

In [3]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession      # at end use sc.stop()

In [4]:
conf = pyspark.SparkConf().setAppName('playing_with_rdd').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [5]:
file_path = "cars.csv"
data_rdd = sc.textFile(file_path)

In [8]:
data_rdd.take(5)

['MAKE,FUELTYPE,ASPIRE,DOORS,BODY,DRIVE,CYLINDERS,HP,RPM,MPG-CITY,MPG-HWY,PRICE',
 'subaru,gas,std,two,hatchback,fwd,four,69,4900,31,36,5118',
 'chevrolet,gas,std,two,hatchback,fwd,three,48,5100,47,53,5151',
 'mazda,gas,std,two,hatchback,fwd,four,68,5000,30,31,5195',
 'toyota,gas,std,two,hatchback,fwd,four,62,4800,35,39,5348']

In [11]:
columns = data_rdd.take(1)[0].split(",")

In [12]:
columns

['MAKE',
 'FUELTYPE',
 'ASPIRE',
 'DOORS',
 'BODY',
 'DRIVE',
 'CYLINDERS',
 'HP',
 'RPM',
 'MPG-CITY',
 'MPG-HWY',
 'PRICE']

In [15]:
data = data_rdd.map(lambda line : line.split(","))
type(data)

pyspark.rdd.PipelinedRDD

In [17]:
make_hp_rdd = data_rdd.map(lambda y : (y.split(",")[0],y.split(",")[7]))
make_hp_rdd.take(4)

[('MAKE', 'HP'), ('subaru', '69'), ('chevrolet', '48'), ('mazda', '68')]

In [18]:
header = data_rdd.first()
rdd_without_header = data_rdd.filter(lambda x : x != header)
rdd_without_header.take(5)

['subaru,gas,std,two,hatchback,fwd,four,69,4900,31,36,5118',
 'chevrolet,gas,std,two,hatchback,fwd,three,48,5100,47,53,5151',
 'mazda,gas,std,two,hatchback,fwd,four,68,5000,30,31,5195',
 'toyota,gas,std,two,hatchback,fwd,four,62,4800,35,39,5348',
 'mitsubishi,gas,std,two,hatchback,fwd,four,68,5500,37,41,5389']

In [20]:
top = make_hp_rdd.first()
make_hp_rdd2 = make_hp_rdd.filter(lambda x : x != top)
make_hp_rdd3 = make_hp_rdd2.mapValues(lambda x : (x,1))
make_hp_rdd3.take(3)

[('subaru', ('69', 1)), ('chevrolet', ('48', 1)), ('mazda', ('68', 1))]

In [36]:
make_hp_rdd4 = make_hp_rdd3.reduceByKey(lambda (value_1, value_2) : int(value_1[0])+int(value_2[0]), int(value_1[1])+int(value_2[1] ))

In [29]:
rdd = sc.parallelize([(13, 'D'), (14, 'T'), (32, '6'), (45, 'T'), (47, '2'), (48, '0'), (49, '2'), (50, '0'), (51, 'T'), (53, '2'), (54, '0'), (13, 'A'), (14, 'T'), (32, '6'), (45, 'A'), (47, '2'), (48, '0'), (49, '2'), (50, '0'), (51, 'X')]) 

In [34]:
from collections  import Counter

rdd.groupByKey().mapValues(Counter).collect()

[(13, Counter({'D': 1, 'A': 1})),
 (14, Counter({'T': 2})),
 (32, Counter({'6': 2})),
 (45, Counter({'T': 1, 'A': 1})),
 (47, Counter({'2': 2})),
 (48, Counter({'0': 2})),
 (49, Counter({'2': 2})),
 (50, Counter({'0': 2})),
 (51, Counter({'T': 1, 'X': 1})),
 (53, Counter({'2': 1})),
 (54, Counter({'0': 1}))]